# Assignment #1: Inertial Observers

* This is Assignment#1 on Classical Mechanics. It is intended to get you to relate our standard notions of physical space and time to the laws of Galilean Galilei.

* Save this file in your Institutional G-Suite google drive and rename it starting with your index number. For example if you are E/90/512 then rename the file to 
'E90512_Assignment#1_InertialObservers.ipynb'

* Get started!

* Once complete share with smaithri@eng.pdn.ac.lk and chanakatb@eng.pdn.ac.lk

* Submit the link at the end of Lesson 1: 'Lesson on Galilean Space Time and Inertial Observers'

* Upload the file through the Assignment Submission Link at the end of the Lesson.

# Setting Up

In [ ]:
import numpy as np
import scipy as sp
from scipy.integrate import odeint
import math
from numpy import linalg
import sympy

from sympy import symbols
from sympy import *

import plotly.graph_objects as go
import plotly.express as px

In [ ]:
class mugas_rigid_body_functions:
  
  def __init__(self): 
    self=[];

  def simulateDySystem(self, dynamicSystemModel, Tmax, dT, X0, sysPara, figTitle, xLabel, yLabel):
    t = np.linspace(0, Tmax, int(Tmax/dT+1))
    sol = odeint(dynamicSystemModel, X0, t, args=(sysPara,));
    self.soln=sol;
    fig = go.Figure()
    [nt,nS]=sol.shape;

    # Add traces
    for nn in range(nS):
      fig.add_trace(go.Scatter(x=t, y=sol[:,nn], mode='lines+markers', name='x'+str(nn+1)))
      fig.update_layout(title=figTitle, xaxis=dict(title=xLabel),yaxis=dict(title=yLabel))
    
    fig.show()
    return [t,sol,fig] 

  def hatMatrix(self, X):
    return np.array([[0., -X[2], X[1]],[X[2], 0., -X[0]],[-X[1], X[0], 0.]])

  def qFromAxisAngles(self, theta, unitAxis):
    return np.concatenate(([np.cos(theta/2)],np.sin(theta/2)*np.array(unitAxis)))

  def RfromQuaternions(self,q):
    q0=q[0]; w=q[1:];
    R=np.identity(3)+2*q0*self.hatMatrix(w)+2*self.hatMatrix(w) @ self.hatMatrix(w);
    return R

  def rotationMatrix2EulerAngles(self,R):
    if R[2,2] <1:
      if R[2,2] > -1:
        theta2=math.acos(R[2,2]);
        theta1=math.atan2(R[0,2],R[1,2]);
        theta3=math.atan2(R[2,0],-R[2,1]);
      else: #R[2,2]=-1 No Unique solution 
        theta2=np.pi;
        theta1=-math.atan2(R[0,1],-R[0,0]);
        theta3=0;
    else: # R[2,2] = +1 No Unique solution
      theta2 = 0;
      theta1 = math.atan2(R[0,1],R[0,0]) ;
      theta3 = 0;
    return [np.pi-theta1,theta2,np.pi-theta3]

  def Re3equalsgamma(self,gamma):
    theta=math.acos(gamma[2]);
    n1=-gamma[1]/np.sin(theta);
    n2=gamma[0]/np.sin(theta);
    q00=self.qFromAxisAngles(theta,[n1,n2,0]);

    return self.RfromQuaternions(q00)

  def rotate_and_translate(self,objectVertices,R,b):
    #object vertices should be given as an numpy array of shape (3,n) 
    #Ex - X=[0, 0, 2, 2, 0, 0, 2, 2], Y=[0, 2, 2, 0, 0, 2, 2, 0], Z=[0, 0, 0, 0, 1, 1, 1, 1]; objectVertices=[X,Y,Z]
    return np.array([[b[0],b[1],b[2]]]).T + R @ objectVertices

  def rigid_body_simulation(self, dt,Tmax,parameters, centerOfMass, M,II,ICq,ICOmega,ICb,ICv):
    invII=linalg.inv(II);
    timeSteps=np.arange(0,Tmax+dt,dt);
    R=self.RfromQuaternions(ICq);
    Omega=ICOmega;
    spPi=R @ II @Omega;
    b=ICb;
    v=ICv;
    p=M*v;

    Eout=[[R,b]];
    Omegaout=[Omega];
    vOut=[v];
    for t in timeSteps:
      if linalg.norm(Omega)>=0.0001:
        nOmega=Omega/linalg.norm(Omega);
        thetaOmegat=dt*linalg.norm(Omega);
      else:
        nOmega=np.array([0,0,0]);
        thetaOmegat=0;

      qOmegat=np.concatenate(([np.cos(thetaOmegat/2)],np.sin(thetaOmegat/2)*nOmega));
      #print(qOmega)
      R=R @ self.RfromQuaternions(qOmegat);
      b=b+dt*v;
      [T,f]=forceModel(self, parameters, centerOfMass, M,II,R,b,Omega,v);
      tau=R @ T; #-hatMatrix(R @ [1.0,1.0,.5]) @ [0,0,1];
      spPi=spPi+dt*(tau);
      Omega=invII @ R.T @ spPi;
      p=p+dt*f
      v=p/M;

    
      Eout+=[[R,b]]; #np.concatenate((Rout,R));
      Omegaout+=[Omega]; #np.concatenate((Omegaout,Omega));
      vOut+=[v];
    
    return [Eout,Omegaout,vOut]

  def addOrthNormFrame(self, fig, o, R, axisRange, axisColor):
    e1=[1,0,0]; e2=[0,1,0]; e3=[0,0,1];
    b1=R @ e1; b2=R @ e2; b3=R @ e3;
    fig.add_trace(go.Scatter3d(x=[str(o[0]),str(o[0]+b1[0])], y=[str(o[1]),str(o[1]+b1[1])], z=[str(o[2]),str(o[2]+b1[2])], hoverinfo='x+y+z', mode='lines', line=dict(width=8, color=axisColor)))
    fig.add_trace(go.Scatter3d(x=[str(o[0]),str(o[0]+b2[0])], y=[str(o[1]),str(o[1]+b2[1])], z=[str(o[2]),str(o[2]+b2[2])], hoverinfo='x+y+z', mode='lines', line=dict(width=8, color=axisColor)))
    fig.add_trace(go.Scatter3d(x=[str(o[0]),str(o[0]+b3[0])], y=[str(o[1]),str(o[1]+b3[1])], z=[str(o[2]),str(o[2]+b3[2])], hoverinfo='x+y+z', mode='lines', line=dict(width=8, color=axisColor)))
    fig.update_layout(showlegend=False, scene=dict(xaxis=dict(range=axisRange[0], autorange=False), yaxis=dict(range=axisRange[1], autorange=False), zaxis=dict(range=axisRange[2], autorange=False), aspectratio=dict(x=1, y=1, z=1)))
    return fig

  def cube_vertices(self,cubeDimensions):
    l=cubeDimensions[0]; w=cubeDimensions[1]; h=cubeDimensions[2];
    X=[0, 0, l, l, 0, 0, l, l]; Y=[0, w, w, 0, 0, w, w, 0]; Z=[0, 0, 0, 0, h, h, h, h]; XX=[X,Y,Z];
    #X=[-l, -l, l, l, -l, -l, l, l]; Y=[-w, w, w, -w, -w, w, w, -w]; Z=[-h, -h, -h, -h, h, h, h, h]; XX=[X,Y,Z]; 
    return XX

  def simulating_a_cube(self, dt, Tmax, cubeDimensions, parameters, centerOfMass, M,II,ICq,ICOmega,ICb,ICv):
    XX=self.cube_vertices(cubeDimensions);
    
    [Es,bs,vs]=self.rigid_body_simulation(dt, Tmax, parameters, centerOfMass, M,II,ICq,ICOmega,ICb,ICv);
    R00=self.Re3equalsgamma(centerOfMass)
    XX=R00.T @ XX;

    ICR=self.RfromQuaternions(ICq);
    XX0=ICR @ XX;
    #qR00=qFromAxisAngles(np.pi/2,np.array([0,1,0]));
    #R00=RfromQuaternions(qR00);
    

    rotatedVertices=[[XX0]]
    for E in Es:
      R=E[0]; 
      b=E[1];
      XXi=qq.rotate_and_translate(XX,R,b);
      XX0=XXi;
      rotatedVertices+=[[XX0]];
    return rotatedVertices

  def animate_particle_motion(self, XX, axisRange, figTitle):
    xx=[jj[0] for jj in XX]
    yy=[jj[1] for jj in XX]
    zz=[jj[2] for jj in XX]

    fig = go.Figure(
        data=[go.Scatter3d(x=[xx[0]], y=[yy[0]],z=[zz[0]],
                        mode="markers",
                        marker=dict(color="red", size=10)),
              go.Scatter3d(x=xx, y=yy,z=zz, name='Path',
                        mode="lines",
                        line=dict(color="blue", width=2))],
        layout=go.Layout(
            scene = dict(
                        xaxis=dict(range=axisRange[0], autorange=False),
                        yaxis=dict(range=axisRange[1], autorange=False),
                        zaxis=dict(range=axisRange[2], autorange=False),
                        aspectratio=dict(x=1, y=1, z=1),
            ),
            title_text=figTitle, hovermode="closest",
            updatemenus=[dict(type="buttons",
                              buttons=[dict(label="Play",
                                            method="animate",
                                            args=[None])])]),
        frames=[go.Frame(
            data=[go.Scatter3d(
                x=[p[0]],
                y=[p[1]],
                z=[p[2]],
                name='Particle',
                mode="markers",
                marker=dict(color="red", size=10))])

            for p in XX]
    )

    fig.show()
    return fig

  def anmated_cube_flat_shading(self, cubeVertices):
    fig = go.Figure(
        frames=[go.Frame(data=[
          go.Mesh3d(
            # 8 vertices of a cube
            x=xx[0][0],
            y=xx[0][1],
            z=xx[0][2],
            # i, j and k give the vertices of triangles
            i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
            j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
            k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
            name='y',
            opacity=0.6,
            color='#DC143C',
            flatshading = True)]) for xx in cubeVertices])

    fig.add_trace(go.Mesh3d(
            # 8 vertices of a cube
            x=cubeVertices[0][0][0],
            y=cubeVertices[0][0][1],
            z=cubeVertices[0][0][2],
            # i, j and k give the vertices of triangles
            i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
            j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
            k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
            name='y',
            opacity=0.6,
            color='#DC143C',
            flatshading = True)
        )

    fig.update_layout(
          title='Animated Cube',
          width=600,
          height=600,
          scene=dict(
                      xaxis=dict(range=[-5., 5.], autorange=False),
                      yaxis=dict(range=[-5., 5.], autorange=False),
                      zaxis=dict(range=[-5., 5.], autorange=False),
                      aspectratio=dict(x=1, y=1, z=1),
                      ),
          updatemenus=[dict(type="buttons",
                              buttons=[dict(label="Play",
                                            method="animate",
                                            args=[None])])])
    len(fig.frames)
    fig.show()
    return fig


In [ ]:
mr=mugas_rigid_body_functions()

# Q4 - A point $P$ in space has the Euclidean representations $P\equiv (1,1,1)$ with respect to some inertial frame $\mathbf{e}$ with origin coinciding with a point $O$ in space. Find the angle that $OP$ makes with the plane spanned by the $\mathbf{e}_1$, $\mathbf{e}_2$ directions. Verify your answer and using plotly plot an orthonormal frame and the point $P$ in that frame.


In [ ]:
P=[1,1,1]; Q=[1,1,0]; O=[0,0,0];
PQ=np.array(P)-np.array(Q)
lengthPQ=linalg.norm(PQ)
anglePQ=np.arccos(np.array(P).dot(np.array(Q))/(linalg.norm(P)*linalg.norm(Q)))
print(['length='+str(lengthPQ), 'angle='+str(180*anglePQ/np.pi)])

['length=1.0', 'angle=35.26438968275467']


In [ ]:
#Adding a Frame
fig = go.Figure()
fig=mr.addOrthNormFrame(fig,O,np.array([[1,0,0],[0,1,0],[0,0,1]]),[[-2,2],[-2,2],[-2,2]],'blue')
fig.show()

In [ ]:
#Adding the Points
fig.add_trace(go.Scatter3d(x=[O[0],P[0]], y=[O[1],P[1]], z=[O[2],P[2]], hoverinfo='x+y+z', mode='lines', line=dict(width=8)))
fig.add_trace(go.Scatter3d(x=[O[0],Q[0]], y=[O[1],Q[1]], z=[O[2],Q[2]], hoverinfo='x+y+z', mode='lines', line=dict(width=8)))

# Q5 - **(Extra Credit Question)** Let $\mathbf{e}$ and $\mathbf{e}'$ be two inertial observers and let $A$ be some space-time event. Let the quadruple $(t,x)\in \mathbb{R}^4$, where $t\in \mathbb{R}$ and  $x\in \mathbb{R}^3$, be the representation of the space-time event $A$ that corresponds to $\mathbf{e}$ while let  $(\tau,\xi)\in \mathbb{R}^4$ where $\tau\in \mathbb{R}$ and  $\xi\in \mathbb{R}^3$ be the representation of the space-time event $A$ that corresponds to $\mathbf{e}'$. 
When comparing the motion described in the two frames we need to know how the two representations (coordinates) are related to each other. Specifically we will show that inertial observers must necessarily be translating at constant velocity with respect to each other without rotations. We do this by showing the following:

* The assumption that time is homogeneous and that all intervals of time are inertial observer invariant means that necessarily $\tau=t+a$ where $a$ is a constant.

* Homogeneity of space implies that necessarily  $\xi=\beta t+R x$ where $\beta$ is a constant $3\times 1$ matrix and $R$ is a constant $3\times 3$ matrix. 

* The assumption that space intervals are inertial observer independent implies that $R$ is an orthonormal constant transformation (that is $R^TR=RR^T=I$).

* Let $O'$ be the origin of the orthonormal frame used by $\mathbf{e}'$ to make spatial measurements. If the space-time event $O'$ has the representation $(t,o)$ according to the observer $\mathbf{e}$ then since $v=\dot{o}=-R^{T}\beta=\mathrm{constant}$ we see that the velocity of the $\mathbf{e}'$ frame with respect to the $\mathbf{e}$ given by $v=\dot{o}$ must be a constant.  If both clocks of $\mathbf{e}$ and $\mathbf{e}'$ are synchronized (that is $a=0$) and if a certain space-time event $A$ has the representation $(t,x)$ according to $\mathbf{e}$ then the space-time event $A$ has the representation $(t,R(x-vt))$ according to $\mathbf{e}'$.


## Sample Answer: The relationship between two inertial observers



The construction of the orthonormal frame and the use of the clock allows an observer $\mathbf{e}$ to assign the ordered quadruple $(t,x)\in \mathbb{R}^4$ where $t\in \mathbb{R}$ and  $x\in \mathbb{R}^3$ to a space-time event in a unique way. A different measurement system, $\mathbf{e}'$ may provide a different identification  $(\tau,\xi)\in \mathbb{R}^4$ where $\tau\in \mathbb{R}$ and  $\xi\in \mathbb{R}^3$ for space time events.

Let $A$ be some some such space-time event. Let the quadruple $(t,x)\in \mathbb{R}^4$ be the representation of the space-time event $A$ in $\mathbf{e}$ while let  $(\tau,\xi)\in \mathbb{R}^4$ be the representation of the space-time event $A$ in $\mathbf{e}'$. 

When comparing the motion described in the two frames we need to know how the two representations (coordinates) are related to each other. Specifically we will provide a sketch of a proof that inertial observers must necessarily be translating at constant velocity with respect to each other without rotations. We do this by showing the following:

* The assumption that time is homogeneous and that all intervals of time are inertial observer invariant means that necessarily $\tau=t+a$ where $a$ is a constant.

* Homogeneity of space implies that necessarily  $\xi=\beta t+R x$ where $\beta$ is a constant $3\times 1$ matrix and $R$ is a constant $3\times 3$ matrix. 

* The assumption that space intervals are inertial observer independent implies that $R$ is an orthonormal constant transformation (that is $R^TR=RR^T=I$).

* Let $O'$ be the origin of the orthonormal frame used by $\mathbf{e}'$ to make spatial measurements. If the space-time event $O'$ has the representation $(t,o)$ according to the observer $\mathbf{e}$ then since $v=\dot{o}=-R^{T}\beta=\mathrm{constant}$ we see that the velocity of the $\mathbf{e}'$ frame with respect to the $\mathbf{e}$ given by $v=\dot{o}$ must be a constant.  If both clocks of $\mathbf{e}$ and $\mathbf{e}'$ are synchronized (that is $a=0$) and if a certain space-time event $A$ has the representation $(t,x)$ according to $\mathbf{e}$ then the space-time event $A$ has the representation $(t,R(x-vt))$ according to $\mathbf{e}'$.


####A rough justification of the above claims 

When comparing the motion described by the two observers we need to know how the two representations (coordinates) are related to each other.  That is we must find the functions $\tau(t,x)$ and $\xi(t,x)$.
The homogeneity assumption of space-time implies that 
\begin{align}
\tau(t_1+T,x_1+a)-\tau(t_2+T,x_2+a)&=\tau(t_1,x_1)-\tau(t_2,x_2)\\
\xi(t_1+T,x_1+a)-\xi(t_2+T,x_2+a)&=\xi(t_1,x_1)-\xi(t_2,x_2)
\end{align}
for all $a, T$ and $t_1,t_2,x_1,x_2$. This necessarily implies that 
\begin{align}
\tau&=a+b t+c x,\\
\xi&=\gamma+\beta t+R x
\end{align}
where $a,b,c,\gamma, \beta, R$ are all constant. Here $a,b\in \mathbb{R}$ and $c,\gamma, \beta\in \mathbb{R}^3$ and $R$ is a $3\times 3$ matrix.
 

The assumption that time is independent of space implies that $c=0$ and the assumption that all inertial observers see the same intervals of time means that necessarily  $b=1$ and hence that $\tau=t+a$. Hence all inertial observers measure time up to an ambiguity of an additive constant and thus  without loss of generality we may assume that all observers have synchronized their clocks and hence that $a=0$. This also implies that a *universal clock* exists. 


The assumption that space intervals are inertial observer independent implies that, $||\xi(t,x_1)-\xi(t,x_2)||=||x_1-x_2||$. Thus $||R (x_1-x_2)||=||x_1-x_2||$ for all $x_1, x_2$. Thus necessarily $R$ must be an orthogonal\footnote{A matrix that satisfies the properties $R^TR=RR^T=I$ is called an orthogonal transformation.} constant transformation. 

Since the space is observed to be homogeneous by all inertial observers without loss of generality we may choose $\gamma=0$ (note that choosing $\gamma=0$ amounts to assuming that the origin of the spatial frames of both observers coincide at the time instant $t=0$ and does not sacrifice any generality since the space is homogeneous we can parallel translate the frames until they coincide at the time instant $t=0$).  Thus we see that the representation of the same space-time event by two different inertial observers are related by 
\begin{align}
(\tau,\xi)=(t,\beta t+Rx).
\end{align}

Since in the following sections we will see that the orthonormal frames are related to each by such an orthogonal transformation it follows from $\xi=Rx$ that the frame used by $\mathbf{e}'$ to make spatial measurements is also an orthonormal frame.
Let $O'$ be the origin of the orthonormal frame used by $\mathbf{e}'$. If the space-time event $O'$ has the representation $(t,o)$ according to the observer $\mathbf{e}$, it has the representation $(t,\beta t+ Ro)=(t,0)$ according to the observer $\mathbf{e}'$. Thus we have that $\beta=-R\dot{o}=-Rv$ where $v=\dot{o}$ and hence that the velocity of the center of the $\mathbf{e}'$ frame with respect to the $\mathbf{e}$ frame, given by $v=\dot{o}=-R^T\beta$, must be a constant. **That is we see that all inertial observers must necessarily be translate at constant velocity with respect to each other without rotation.**

This also shows that the representation of a space-time event denoted by $(t,x)$ according to $\mathbf{e}$ must necessarily have the representation $\left(t,R(x-vt)\right)$ for some constant $v\in \mathbb{R}^3$ according to any other inertial frame $\mathbf{e}'$. **Space appears to be homogeneous only for such observers.** In particular we can see that this is not the case for observers rotating with respect to an inertial observer $\mathbf{e}$. That is a rotating observer will not observe space to be homogeneous (Show that this is true.). 

Since $R$ is a constant, without loss of generality, one can always pick the orthonormal frame used by $\mathbf{e}'$ to be parallel to the one used by $\mathbf{e}$ so that $R=I_{3\times 3}$. Then we see that $\xi(t)=x(t)-vt$ in parallel translating inertial frames. **It is traditional to refer to parallel frames that translate at constant velocities with respect to each other as {inertial frames}.**

# Q6 -  An inertial observer $\mathbf{e}$ notices that an object is moving in a straight line at a constant velocity. Show that all other inertial observers will also agree that the object is moving in a straight line at a constant velocity.

# Q7 - An inertial observer $\mathbf{e}$ notices that an object is moving in a straight line at a constant acceleration (for example such as the free falling object under gravity). Show that other inertial observers need not agree that the object is moving in a straight line.


# Answers to Q6 and Q7

Let $\mathbf{e}$ and $\mathbf{e}'$ represent two inertial observers with $\mathbf{e}'$ moving  parallel to $\mathbf{e}$ at a constant velocity $v=[v_1\:\:\:v_2\:\:\:v_3]^T$.

If $(t,x)$ is a representation of a point $P$ in $\mathbf{e}$ we have seen in Q5 that this same point has the representation $\left(t,(x-vt)\right)$ in the $\mathbf{e}'$ frame.

If the point $P$ is seen to be moving in a straight line by $\mathbf{e}$ (without loss of generality say a straight line through the origin) then $x(t)=f(t)\,a$ where $a=[a_1\:\:\:a_2\:\:\:a_3]$ is a constant and $f(\cdot)$ is some function of time. Then this point has the representation $\left(t,(f(t)\,a-vt)\right)$ in the $\mathbf{e}'$ frame.

## Answer to Q6

If $P$ is observed in $\mathbf{e}$ to move in a straight line at a constant velocity then $f(t)\equiv t$ and then in the $\mathbf{e}'$ frame this point has the representation $\left(t,t(a-v)\right)$ and hence is observed to be moving in a straight line as well.

In [ ]:
a=np.array([1,0,0]);
v=np.array([0,0,1])
tt=np.linspace(0,5,101)
zeta=np.zeros((len(tt),3));
for i,t in enumerate(tt):
  zeta[i,:]=t*(a-v);

In [ ]:
fig=mr.animate_particle_motion(zeta,[[-3,3],[-3,3],[-3,3]],'Motion in the e-prime frame')

## Answer to Q7

If $P$ is observed in $\mathbf{e}$ to move in a straight line at a constant acceleration then $f(t)\equiv t^2$ and then in the $\mathbf{e}'$ frame this point has the representation $\left(t,t(at-v)\right)$ and hence is observed to be moving in a **curved** path.

In [ ]:
a=np.array([0,0,-1]);
v=np.array([1,0,0])
tt=np.linspace(0,5,101)
zeta=np.zeros((len(tt),3));
for i,t in enumerate(tt):
  zeta[i,:]=t*(t*a-v);

In [ ]:
fig=mr.animate_particle_motion(zeta,[[-3,3],[-3,3],[-3,3]],'Motion in the e-prime frame')